# **CH01.2. GloVe**

#### **`Paper Info`** : GloVe: Global Vectors for Word Representation
##### $ \hspace{1.75cm} - $ Jeffrey Pennington, Richard Socher, Christopher D. Manning et al. (EMNLP/2014)

<b></b>

> ## **요약(Summary)**

| Item | Description |
|------|-------------|
| Research topic | 전역 co-occurrence 통계를 활용한 log-bilinear 회귀로 단어 분산 표현을 학습하는 GloVe 모델 제안 |
| Key findings | $ \cdot{} $ 확률 비율 ratio 기반 직관으로 linear substructure를 유도해 analogy에 강점 확보 <br> $ \cdot{} $ 가중 최소제곱 회귀와 weighting function으로 희귀 단어와 초빈 단어를 균형 있게 학습 <br> $ \cdot{} $ Word analogy, Word similarity, NER에서 기존 count 기반과 prediction 기반 대비 우수 성능 |
| Contributions | $ \cdot{} $ count based와 predictive 접근을 이론적으로 연결하는 목적함수 제안 <br> $ \cdot{} $ 교환 대칭 word context를 보존하는 bias 포함 log bilinear 모델화 <br> $ \cdot{} $ 대규모 말뭉치에서 효율적 학습과 재현 가능한 설정 제시 |


<b></b>

> ## **연구 배경(Motivation & Prior Work)**

#### **(1) 기존 연구 및 한계점** :
##### $ \hspace{0.15cm} \cdot{} $ LSA(Latent Semantic Analysis) 등 count-based 방법은 전역 통계를 활용하지만 단어 간 선형 의미 관계를 반영하지 못함
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram, CBOW 등 predict-based 방법은 의미 관계를 잘 학습하지만 전역 통계 정보 활용이 제한적임

##### **(`PLUS`) 선형 의미 관계(Linear Semantic Relationship)** : 단어들 간의 의미적 관계가 벡터 공간에서 선형 연산(덧셈, 뺄셈)으로 표현될 수 있는 관계

#### **(2) 연구 목표** :
##### $ \hspace{0.15cm} \cdot{} $ 전역 통계 활용 효율성과 예측 기반 선형 의미 구조를 동시에 달성
##### $ \hspace{0.15cm} \cdot{} $ 단어 의미 차이를 조건부 동시출현 확률의 비율로 모델링
##### $ \hspace{0.15cm} \cdot{} $ 단순하고 해석 가능한 목적식을 통해 의미적 규칙성을 반영

#### **(3) 제안된 방법론** :
##### $ \hspace{0.15cm} \cdot{} $ 단어 쌍의 log co-occurrence를 임베딩 내적과 bias의 합으로 근사함
##### $ \hspace{0.15cm} \cdot{} $ weighted least squares 목적함수를 사용해 빈도 불균형을 완화함
##### $ \hspace{0.15cm} \cdot{} $ 학습된 target과 context 벡터의 합을 최종 단어 임베딩으로 구성함

<b></b>

> ## **방법론(Method)**

#### **(0) Co-occurrence** :
##### $ \hspace{0.15cm} \cdot{} $ Co-occurrence matrix : corpus 내에서 단어 쌍 $ \, (w_{i},w_{j}) $가 같은 (문맥) 윈도우 안에서 함께 등장한 빈도(횟수)를 집계한 행렬 
##### $ \hspace{0.3cm} \Rightarrow{} X_{ij} := \displaystyle{} \sum_{t=1}^{T} \sum_{\substack{-c \le{} d \le{} c \\ d \neq{} 0}} \mathbb{I}\{w_{t}=i, w_{t+d}=j\} $
##### $ \hspace{0.45cm} \text{where } \, \mathbb{I}\{\cdot{}\} : \text{indicator function}, \;\; c : \text{window radius}, \;\; T : \text{token length} $
##### $ \hspace{0.15cm} \cdot{} $ Co-occurence (conditional) probability : 중심 단어 $ w_{i} $가 주어졌을 때 문맥 단어 $ w_{j} $가 같은 윈도우 내에 등장할 확률
##### $ \hspace{0.3cm} \Rightarrow{} P(w_{j}|w_{i}) = \displaystyle{} \frac{X_{ij}}{X_{i}} \;\; \text{ where } \, X_{i} = \displaystyle{} \sum^{V}_{k=1} X_{ik} $ 
##### $ \hspace{0.15cm} \cdot{} $ Co-occurrence ratio : 두 단어 $ w_{i} $와 $ w_{j} $가 특정 단어 $ w_{k} $에 대해 갖는 co-occurrence probability의 비율
##### $ \hspace{0.3cm} \Rightarrow{} \displaystyle{} \frac{P(w_{k}\mid{}w_{i})}{P(w_{k}\mid{}w_{j})} = \frac{X_{ik}/X_{i}}{X_{jk}/X_{j}} $

##### **(`PLUS`)** Co-occurrence ratio는 임베딩 공간에서 벡터 차이를 표현하기 위해  $ F((\mathbf{w}_{i}-\mathbf{w}_{j})^{\top{}}\tilde{\mathbf{w}}_{k}) $ 형태로 매핑됨
##### $ \hspace{0.45cm} \Rightarrow{} \displaystyle{} \frac{P(w_{k}\mid{}w_{i})}{P(w_{k}\mid{}w_{j})} = \exp{}\bigg({}( \mathbf{w}_{i}-\mathbf{w}_{j} )^{\top{}}\tilde{\mathbf{w}}_{k}\bigg){} $

#### **(1) GloVe(Global Vectors for Word Representation)** :
##### $ \hspace{0.15cm} \cdot{} $ Log co-occurence conditional probability $ \, \log{} \big({} P(w_{j}|w_{i}) \big){} $를 log linear 모델로 가정(근사)
##### $ \hspace{0.3cm} \Rightarrow{} \log{} \big({} P(w_{j}|w_{i}) \big){} \approx{} \mathbf{w}_{i}^{\top{}}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j} - \log{} Z_{i} $
##### $ \hspace{0.45cm} \text{where } \, X_{i} = \displaystyle{} \sum^{V}_{k=1} X_{ik}, \;\; \mathbf{w}_{i},\tilde{\mathbf{w}}_{j}\in{}\mathbb{R}^{d} : \text{target, context embeddings} $
##### $ \hspace{0.45cm} \text{and } \, b_{i},\tilde{b}_{j}\in{}\mathbb{R} : \text{bias terms}, \;\; Z_{i}: \text{normalizer} $
##### $ \hspace{0.15cm} \cdot{} $ Weighted Least Squares Error 문제로 손실 함수를 정의
##### $ \hspace{0.3cm} \Rightarrow{} J^{*} \approx{} \displaystyle{} \argmin_{\theta{}} \sum_{i=1}^{V}\sum_{j=1}^{V} f(X_{ij})\Big({}\mathbf{w}_{i}^{\top{}}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j} - \log X_{ij}\Big){}^{2} $
##### $ \hspace{0.45cm} \text{where } \, f(x)=\begin{cases}(x/x_{\max{}})^{\alpha{}} & x<x_{\max{}} \\ 1 & \text{otherwise}\end{cases} \; : \text{weighted function} $

##### **(`WHY?`)**
##### $ \hspace{0.3cm} \cdot{} $ log-linear 모델로 조건부 확률의 로그를 내적과 바이어스로 근사함을 가정
##### $ \hspace{0.45cm} \Rightarrow{} \log{} \big({} P(w_{j}\mid{}w_{i}) \big){} = \log{} (\frac{X_{ij}}{X_{i}})\approx{} \mathbf{w}_{i}^{\top{}}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j} - \log{} Z_{i} $
##### $ \hspace{0.45cm} \Leftrightarrow{} \log{} (X_{ij}) - \log{} (X_{i}) \approx{} \mathbf{w}_{i}^{\top{}}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j} - \log{} (Z_{i}) $
##### $ \hspace{0.3cm} \cdot{} $ 이항하여 $ \, \log{} (X_{ij}) $를 target variable로 하는 선형 회귀 형태로 변환
##### $ \hspace{0.45cm} \Rightarrow{} \log{} (X_{ij}) \approx{} \mathbf{w}_{i}^{\top{}}\tilde{\mathbf{w}}_{j} + b_{i} + \tilde{b}_{j} + \log{} (X_{i}) - \log{} (Z_{i}) $
##### $ \hspace{0.3cm} \cdot{} \, \log{} Z_{i} $와 $ \log{} X_{i} $항은 단어 $ i $에만 의존하므로 바이어스 $ b_{i} $로 흡수
##### $ \hspace{0.45cm} \Rightarrow{} \log{} (X_{ij}) = \mathbf{w}_{i}^{\top{}}\tilde{\mathbf{w}}_{j} + b_{i}^{'} + \tilde{b}_{j} \;\; \text{ where } \, b_{i}^{'} = b_{i} + \log{} (X_{i}) - \log{} (Z_{i}) $
##### $ \hspace{0.3cm} \cdot{} $ Weighted Least Squares Error 정의
##### $ \hspace{0.45cm} \Rightarrow{} \text{MSE} : \displaystyle{} f(X_{ij}) \cdot{} \epsilon{}^{2} \;\; \text{ where } \,  \epsilon{} := \big({} \mathbf{w}_{i}^{\top{}}\tilde{\mathbf{w}}_{j} + b_{i}^{'} + \tilde{b}_{j} \big){} - \log{}(X_{ij}) $
##### $ \hspace{0.45cm} \Leftrightarrow{} J^{*} \approx{} \displaystyle{} \argmin_{\theta{}} \sum_{i=1}^{V}\sum_{j=1}^{V} f(X_{ij}) \cdot{} \epsilon{}^{2} $

<b></b>

> ## **실험(Experiments)**

#### **(1) 실험 설정** :
##### $ \hspace{0.15cm} \cdot{} $ 데이터셋
| Item | Description |
|------|--------------|
| Training dataset | Wikipedia 2014 + Gigaword 5 + Common Crawl |
| Vocabulary size($V$) | $ 400\text{K} $ / $ 1.9\text{M} $ |
| token size($T$) | $ 6\text{B} $ / $ 42\text{B} $ |
| Evaluation dataset | Google analogy, MSR analogy, WordSim353, MEN |
##### $ \hspace{0.15cm} \cdot{} $ 모델
| Item | Description |
|------|--------------|
| Model type | Global log-bilinear regression (GloVe) |
| Architecture | target, context embeddings + bias |
| Parameter size | $ \approx{} 2VD + 2V$ |
| Hyper parameter | $ \cdot{} $ embedding dimension($ D $) $ = 50/100/200/300$ <br> $ \cdot{} $ window size($ c $) = 10 <br> $ \cdot{} \, x_{\max}=100$, $\alpha=0.75$ |
##### $ \hspace{0.15cm} \cdot{} $ 훈련 및 평가
| Item | Description |
|------|--------------|
| Loss function | Weighted least squares on $\log{} X_{ij}$ |
| Evaluation metric | Analogy accuracy, Spearman correlation |
| Baseline models | Skip-gram, CBOW, LSA (SVD-based) |
| Training hardware | CPU multi-threaded |
| Training time | ? |

#### **(2) 실험 결과** :
##### $ \hspace{0.15cm} \cdot{} $ Word analogy에서 semantic·syntactic 관계 모두에서 높은 정확도 달성 ($ 71.7\% $ at $ 6\text{B}, \; 75\% $ at $ 42\text{B} $)
##### $ \hspace{0.15cm} \cdot{} $ Word similarity에서 Spearman 상관 WS-353($ 75.9 $), MC($ 83.6 $), RG($ 82.9 $)로 강한 상관성을 보임
##### $ \hspace{0.15cm} \cdot{} $ NER(CoNLL-2003) 실험에서 CRF pipeline의 F1 향상($88.3\%$)을 달성함
##### $ \hspace{0.15cm} \cdot{} $ 임베딩 차원이 증가함에 따라 성능은 향상되나 약 $ 200D $ 이후 수익 체감이 발생함
##### $ \hspace{0.15cm} \cdot{} $ 작은 윈도우는 syntactic 관계에, 큰 윈도우는 semantic 관계에 유리함
##### $ \hspace{0.15cm} \cdot{} $ 대규모 코퍼스일수록 전역 의미 정보가 강화되어 semantic accuracy가 상승함
##### $ \hspace{0.15cm} \cdot{} $ Skip-gram, CBOW 대비 시간–정확도 효율성이 높음

<b></b>

> ## **결론(Conclusion)**

#### **(1)** GloVe는 전역 co-occurrence ratio를 내적 선형식으로 모델링하여 의미 관계를 공간적으로 표현함

#### **(2)** weighted least squares 목적함수와 weighting function으로 고빈도·저빈도 단어의 영향을 균형화함

#### **(3)** count-based와 predict-based 접근의 장점을 결합하여 대규모 코퍼스에서도 효율성과 성능을 모두 달성함

<b></b>

> ## **부록(Appendix)**

<b></b>

> ## **비평(Commentary)**

<b></b>

> ## **참고 문헌(Reference)**

#### **(1) 관련 후속 논문** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**

#### **(2) 기타 참고 자료** :
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**
##### $ \hspace{0.15cm} \cdot{} $ **[~]**